## <center> DECAGON TEST </center> 
<hr />
1. Create an ETL workflow that brings the data into an SQL database. 
    - You can use Postgres or MySQL. We plan to have a database that contains all **continents, countries, currencies, and languages. (continents, countries, currencies, languages should all be in a different table)**
    
    

2. With the data in the database, create the following reports using SQL.
    - List all the continents and the total number of countries in each. 
    For example, Africa 100, Europe 10, etc. The continent's name and country
    - List all the languages and commas separated countries that speak thelanguage.
    - List all the countries and the total number of languages spoken.

## <center> WORKSPACE </center> 

<hr />

### DATA SOURCES

In [1]:
# data sources
countries_json = 'https://raw.githubusercontent.com/annexare/Countries/master/data/countries.json'
continent_json = 'https://raw.githubusercontent.com/annexare/Countries/master/data/continents.json'
languages_json = 'https://raw.githubusercontent.com/annexare/Countries/master/data/languages.json'
countries_iso2_to_iso3_json = 'https://raw.githubusercontent.com/annexare/Countries/master/data/countries.2to3.json'

#### Import Necessary Modules

In [2]:
import pandas as pd
from sqlalchemy import create_engine

### EXTRACT & TRANSFORMATION OF DATA `E-T`

#### Writing Function to Perform `ET` on Countries Data

In [3]:
# Extracting and transforming Countries Data
def extract_and_transform_countries_data():
    countries_df = pd.read_json(countries_json)

    countries_df = countries_df.T

    countries_df.reset_index(inplace = True)

    countries_df.rename(columns = {'index':'iso2Code', 
                                   'native':'nativeName', 
                                   'currency':'currencyCode', 
                                   'languages':'languageCode',
                                   'phone':'phoneCode',
                                   'continent':'continentCode'}, inplace = True)
    
    countries_df['currencyCode'] = [','.join(i) if isinstance(i, list) else i for i in countries_df['currencyCode']]

    countries_df['languageCode'] = [','.join(i) if isinstance(i, list) else i for i in countries_df['languageCode']]

    countries_df['phoneCode'] = [i[0] for i in countries_df['phoneCode']]
    
    countries_df.drop(['continents'], axis=1, inplace = True)
    
    return countries_df

def extract_and_transform_countries_iso2_to_iso3_data():
    countries_iso2_to_iso3_df = pd.read_json(countries_iso2_to_iso3_json, typ='series')

    countries_iso2_to_iso3_df = countries_iso2_to_iso3_df.to_frame()

    countries_iso2_to_iso3_df.reset_index(inplace = True)

    countries_iso2_to_iso3_df.rename(columns = {'index':'iso2Code',0:'iso3Code' }, inplace = True)
    
    countries_new_df = extract_and_transform_countries_data()
    
    # append iso3Code to countries table from countries_iso2_to_iso3_df
    
    countries_new_df = pd.merge(countries_new_df, countries_iso2_to_iso3_df, on ='iso2Code')
    
    fields = ['name', 'capital','nativeName','iso2Code', 
           'iso3Code', 'phoneCode', 'continentCode','currencyCode','languageCode']
    
    countries_new_df = countries_new_df[fields]
    
    return countries_new_df

#### Testing Function 

In [4]:
extract_and_transform_countries_iso2_to_iso3_data()

,name,capital,nativeName,iso2Code,iso3Code,phoneCode,continentCode,currencyCode,languageCode
0,Andorra,Andorra la Vella,Andorra,AD,AND,376,EU,EUR,ca
1,United Arab Emirates,Abu Dhabi,دولة الإمارات العربية المتحدة,AE,ARE,971,AS,AED,ar
2,Afghanistan,Kabul,افغانستان,AF,AFG,93,AS,AFN,"ps,uz,tk"
3,Antigua and Barbuda,Saint John's,Antigua and Barbuda,AG,ATG,1268,NA,XCD,en
4,Anguilla,The Valley,Anguilla,AI,AIA,1264,NA,XCD,en
...,...,...,...,...,...,...,...,...,...
245,Yemen,Sana'a,اليَمَن,YE,YEM,967,AS,YER,ar
246,Mayotte,Mamoudzou,Mayotte,YT,MYT,262,AF,EUR,fr
247,South Africa,Pretoria,South Africa,ZA,ZAF,27,AF,ZAR,"af,en,nr,st,ss,tn,ts,ve,xh,zu"
248,Zambia,Lusaka,Zambia,ZM,ZMB,260,AF,ZMW,en


#### Writing Function to Perform `ET` on Continents Data

In [5]:
# Extracting and transforming continents Table
def extract_and_transform_continents_data():
    continents_df = pd.read_json(continent_json, typ='series')

    continents_df = continents_df.to_frame()

    continents_df.reset_index(inplace = True)

    continents_df.rename(columns = {'index':'code',0:'name' }, inplace = True)
    
    return continents_df

#### Testing Function 

In [6]:
# Visualize Continent Table
extract_and_transform_continents_data()

,code,name
0,AF,Africa
1,AN,Antarctica
2,AS,Asia
3,EU,Europe
4,NA,North America
5,OC,Oceania
6,SA,South America


#### Writing Function to Perform `ET` on Languages Data

In [7]:
def extract_and_transform_language_data():
    languages_df = pd.read_json(languages_json)

    languages_df = languages_df.T

    languages_df.reset_index(inplace = True)

    languages_df.rename(columns = {'index':'code', 'native':'nativeName'}, inplace = True)
    
    languages_df.drop('rtl',axis=1, inplace = True)

    return languages_df

#### Testing Function 

In [8]:
# Visualize Language Table
extract_and_transform_language_data()

,code,name,nativeName
0,aa,Afar,Afar
1,ab,Abkhazian,Аҧсуа
2,af,Afrikaans,Afrikaans
3,ak,Akan,Akana
4,am,Amharic,አማርኛ
...,...,...,...
180,yi,Yiddish,ייִדיש
181,yo,Yoruba,Yorùbá
182,za,Zhuang,Cuengh / Tôô / 壮语
183,zh,Chinese,中文


#### Writing Function to Perform `ET` on Currency Data

In [9]:
# Extracting and transforming Currency Table
def extract_and_transform_currency_data():
    
    countries_df = extract_and_transform_countries_iso2_to_iso3_data()
    
    currency_data = countries_df[['iso2Code','currencyCode']].copy()

    currency_data = currency_data.explode('currencyCode')

    currency_data = currency_data.explode('currencyCode')

    currency_data = currency_data.groupby('currencyCode')['iso2Code'].apply(list)

    currency_data = currency_data.to_frame()

    currency_data.reset_index(inplace = True)

    currency_data.rename(columns = {'currencyCode':'code','iso2Code':'countryIso2code' }, inplace = True)

    currency_data['countryIso2code'] = [','.join(i) if isinstance(i, list) else i for i in currency_data['countryIso2code']]
    
    currency_data = currency_data.reset_index().rename(columns={'index':'id'})
    
    currency_data = (currency_data.set_index(['id', 'countryIso2code']).apply(lambda x: x.str.split(',').explode()).reset_index())

    currency_data = currency_data[currency_data['code']!='']
    
    currency_data.drop(['id'], axis=1, inplace = True)
    
    currency_data = currency_data.reset_index().rename(columns={'index':'id'})
    
    fields = ['id', 'code', 'countryIso2code']
    
    currency_data = currency_data[fields]
    
    return currency_data

#### Testing Function 

In [10]:
# Visualize Currency Table
extract_and_transform_currency_data()

,id,code,countryIso2code
0,1,AED,AE
1,2,AFN,AF
2,3,ALL,AL
3,4,AMD,AM
4,5,ANG,"CW,SX"
...,...,...,...
177,178,XOF,"BF,BJ,CI,GW,ML,NE,SN,TG"
178,179,XPF,"NC,PF,WF"
179,180,YER,YE
180,181,ZAR,ZA


<hr />

### SCHEMA DEFINITION
- This script below was loaded via **`the terminal`**, using **```mysql -h localhost -u root```***

In [11]:
from __future__ import print_function

import mysql.connector
from mysql.connector import errorcode

DB_NAME = 'decagon_test_db'

TABLES = {}

TABLES['countries'] = (
  "CREATE TABLE IF NOT EXISTS `countries` ("
  "`name` varchar(255) COLLATE utf8mb4_unicode_ci NOT NULL,"
  "`capital` varchar(255) COLLATE utf8mb4_unicode_ci NOT NULL,"
  "`nativeName` varchar(255) COLLATE utf8mb4_unicode_ci NULL,"
  "`iso2Code` varchar(255) NOT NULL,"
  "`iso3Code` varchar(255) NOT NULL,"
  "`phoneCode` varchar(255) COLLATE utf8mb4_unicode_ci NOT NULL,"
  "`currencyCode` varchar(255) COLLATE utf8mb4_unicode_ci NOT NULL,"
  "`continentCode` varchar(255)COLLATE utf8mb4_unicode_ci NOT NULL,"
  "`languageCode` varchar(255)COLLATE utf8mb4_unicode_ci NOT NULL,"
  "PRIMARY KEY (`name`)"
") ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci")


TABLES['continents'] = (
    "CREATE TABLE IF NOT EXISTS `continents` ("
      "`code` varchar(255) NOT NULL,"
      "`name` varchar(255) COLLATE utf8mb4_unicode_ci NOT NULL,"
      "PRIMARY KEY (`code`)"
    ") ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci")


TABLES['currency'] = (
    "CREATE TABLE IF NOT EXISTS `currency` ("
      "`id` int COLLATE utf8mb4_unicode_ci NOT NULL,"
      "`code` varchar(5) COLLATE utf8mb4_unicode_ci NOT NULL,"
      "`countryIso2Code` varchar(255) COLLATE utf8mb4_unicode_ci NULL,"
      "`description` varchar(255) COLLATE utf8mb4_unicode_ci DEFAULT NULL,"
      "`symbol` varchar(5) COLLATE utf8mb4_unicode_ci DEFAULT NULL,"
     " PRIMARY KEY (`id`)"
   " ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci")


TABLES['languages'] = (
    "CREATE TABLE IF NOT EXISTS `languages` ("
      "`code` varchar(255) NOT NULL,"
      "`name` varchar(255) COLLATE utf8mb4_unicode_ci NOT NULL,"
      "`nativeName` varchar(255) COLLATE utf8mb4_unicode_ci NULL,"
      "PRIMARY KEY (`code`)"
    ") ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci")

### CREATE DATABASE (IF IT DOESN'T EXIST)

In [12]:
cnx = mysql.connector.connect(user='root')
cursor = cnx.cursor()

def create_database(cursor):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cursor.execute("USE {}".format(DB_NAME))
except mysql.connector.Error as err:
    print("Database {} does not exists.".format(DB_NAME))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor)
        print("Database {} created successfully.".format(DB_NAME))
        cnx.database = DB_NAME
    else:
        print(err)
        exit(1)

Database decagon_test_db does not exists.
Database decagon_test_db created successfully.


### CREATE TABLES (IF THEY DON'T EXIST)

In [13]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

# cursor.close()
# cnx.close()

Creating table countries: OK
Creating table continents: OK
Creating table currency: OK
Creating table languages: OK


### LOAD DATA INTO TABLES `L`

#### Load Countries Data into Database Table

In [14]:
countries = [tuple(r) for r in extract_and_transform_countries_iso2_to_iso3_data().to_numpy()]

try:
    
    cnx = mysql.connector.connect(user='root')
    
    cursor = cnx.cursor()

    cursor.execute("USE {}".format(DB_NAME))
    
    countries_insert_query = """INSERT INTO countries (name, capital, nativeName,iso2Code, iso3Code, phoneCode, continentCode, currencyCode, languageCode) 
                               VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s) """

    countries_records_to_insert = countries

    #     cursor = connection.cursor()
    cursor.executemany(countries_insert_query, countries_records_to_insert)
    cnx.commit()
    print(cursor.rowcount, "Record inserted successfully into countries table")

except mysql.connector.Error as error:
    print("Failed to insert countries record into MySQL countries table {}".format(error))


250 Record inserted successfully into countries table


#### Load Continents Data into Database Table

In [15]:
continents = [tuple(r) for r in extract_and_transform_continents_data().to_numpy()]

try:
        
    continents_insert_query = """INSERT INTO continents (code, name) 
                               VALUES (%s, %s) """

    continents_records_to_insert = continents

    #     cursor = connection.cursor()
    cursor.executemany(continents_insert_query, continents_records_to_insert)
    cnx.commit()
    print(cursor.rowcount, "Record inserted successfully into continents table")

except mysql.connector.Error as error:
    print("Failed to insert continents record into MySQL continents table {}".format(error))   

7 Record inserted successfully into continents table


#### Load Languages Data into Database Table

In [16]:
languages = [tuple(r) for r in extract_and_transform_language_data().to_numpy()]

try:
        
    languages_insert_query = """INSERT INTO languages (code, name, nativeName) 
                               VALUES (%s, %s, %s) """

    languages_records_to_insert = languages

    #     cursor = connection.cursor()
    cursor.executemany(languages_insert_query, languages_records_to_insert)
    cnx.commit()
    print(cursor.rowcount, "Record inserted successfully into languages table")

except mysql.connector.Error as error:
    print("Failed to insert languages record into MySQL languages table {}".format(error))     

185 Record inserted successfully into languages table


#### Load Currency Data into Database Table

In [17]:
currency = [tuple(r) for r in extract_and_transform_currency_data().to_numpy()]

try:
        
    currency_insert_query = """INSERT INTO currency (id, code, countryIso2Code) 
                               VALUES (%s, %s, %s ) """

    currency_records_to_insert = currency

    #     cursor = connection.cursor()
    cursor.executemany(currency_insert_query, currency_records_to_insert)
    cnx.commit()
    print(cursor.rowcount, "Record inserted successfully into currency table")

except mysql.connector.Error as error:
    print("Failed to insert currency record into MySQL currency table {}".format(error))     

182 Record inserted successfully into currency table


<hr />

### ANSWER TO QUESTIONS

#### Question 1
- List all the continents and the total number of countries in each

In [18]:
QUERY = """select a.name as Continent, count(b.name) as 'Country Count' from continents a left join countries b on a.code = b.continentCode group by a.name"""

cursor.execute(QUERY)
a = cursor.fetchall()

ANSWER1 = pd.DataFrame(a, columns =['Continent', 'Country Count'])
ANSWER1

,Continent,Country Count
0,Africa,58
1,Antarctica,5
2,Asia,52
3,Europe,53
4,North America,41
5,Oceania,27
6,South America,14


#### Question 2
- List all the languages and commas separated countries that speak the language.

In [19]:
QUERY = """
            with language_country_data as (
                SELECT
                  countries.name,
                  SUBSTRING_INDEX(SUBSTRING_INDEX(countries.languageCode, ',', numbers.n), ',', -1) languageCode
                FROM
                  (SELECT 1 n UNION ALL SELECT 2
                   UNION ALL SELECT 3 UNION ALL SELECT 4) numbers INNER JOIN countries
                  ON CHAR_LENGTH(countries.languageCode)
                     -CHAR_LENGTH(REPLACE(countries.languageCode, ',', ''))>=numbers.n-1
                ORDER BY
                  name, n)

                select 
                    b.name as 'Language', 
                    group_concat(a.name) as Countries
                from language_country_data a 
                left join languages b on a.languageCode = b.code
                group by b.name

        """

cursor.execute(QUERY)
a = cursor.fetchall()

ANSWER2 = pd.DataFrame(a, columns =['Language', 'Countries'])
ANSWER2

,Language,Countries
0,None,Antarctica
1,Afrikaans,"Namibia,South Africa"
2,Albanian,"Albania,Kosovo,Montenegro"
3,Amharic,Ethiopia
4,Arabic,"Algeria,Bahrain,Chad,Comoros,Djibouti,Egypt,Er..."
...,...,...
104,Turkmen,"Turkmenistan,Afghanistan"
105,Ukrainian,Ukraine
106,Urdu,"Fiji,Pakistan"
107,Uzbek,"Afghanistan,Uzbekistan"


#### Question 3
- List all the countries and the total number of languages spoken.

In [20]:
QUERY = """SELECT name as Country, LENGTH(languageCode) - LENGTH(REPLACE(languageCode, ',','')) +1 as 'Lng Count' FROM countries"""

cursor.execute(QUERY)
b = cursor.fetchall()

ANSWER2 = pd.DataFrame(b, columns =['Country', 'Lng Count'])
ANSWER2

,Country,Lng Count
0,Afghanistan,3
1,Åland,1
2,Albania,1
3,Algeria,1
4,American Samoa,2
...,...,...
245,Wallis and Futuna,1
246,Western Sahara,1
247,Yemen,1
248,Zambia,1


##### Close Database Connection


In [21]:
if (cnx.is_connected()):
    cursor.close()
    cnx.close()
    print("MySQL connection is closed")

MySQL connection is closed


<hr />

#### PROJECT FLOW

- `Extract` Data from Source, `Transform` and `Load` to MySQL Database
    - Extract Using Python
    - Initial Transform Using Python
    - Loading Using Python with SQL  <br>
- Report Generation using MySQL


##### DISCLAIMER

- In a real-world project, I would enrich these datasets (esp. the language and currency data). <br /> However, this flow was created within the scope of the data provided.

- There are a number of ways I would have fiurther ckeaned the data to make for a great database design, but I'm out of Time Now.